In [74]:
# python
from pathlib import Path

# accessibility
import pyomu

# data analysis
import geopandas as gpd
import pandas as pd

In [3]:
current_dir = Path.cwd()
parent_dir = current_dir.parent

## Ejemplo datos establecimientos Medellin

El siguiente conjunto de datos se toma como ejemplo para conocer el formato adecuado para reunir los datos de establecimientos. Está disponible en le repositorio de la librería pyomu: https://github.com/OMU-LATAM/Accesibilidad/tree/main/notebooks/data

In [145]:
establecimiento_medellin = gpd.read_file(
    parent_dir / "data" / "establecimientos_medellin.geojson"
)

In [6]:
establecimiento_medellin.head()

,id,hex_d,destino,nombre,tipo,oferta,geometry
0,0,8866183403fffff,"6.186777, -75.647701",INSTITUCION EDUCATIVA PRADITO,Educación Inicial,,POINT (-75.64770 6.18678)
1,1,8866183403fffff,"6.18679, -75.640682",SECCION ESCUELA LA VERDE,Educación Inicial,,POINT (-75.64068 6.18679)
2,2,8866183407fffff,"6.185226, -75.654597",SECCION ESCUELA CARLOS BETANCUR BETANCUR,Educación Inicial,,POINT (-75.65460 6.18523)
3,3,8866183407fffff,"6.186229, -75.656151",CENTRO EDUCATIVO MI LINDO CARRUSEL,Educación Inicial,,POINT (-75.65615 6.18623)
4,4,8866183415fffff,"6.181923, -75.642756",COLEGIO COOPERATIVO SAN ANTONIO DE PRADO,Educación Inicial,,POINT (-75.64276 6.18192)


In [7]:
establecimiento_medellin.tipo.value_counts()

Educación Inicial       1260
Educación Primaria       988
Educación Secundaria     720
Salud                    133
Name: tipo, dtype: int64

## Datos establecimientos Cuenca

A continuación se realiza el procesamiento de la información de Cuenca para acomodar los datos disponibles de acuerdo al forma establecido en el ejemplo anterior.

### Educación

In [124]:
educacion_cuenca = gpd.read_file(
    parent_dir / "data" / "cuenca" / "equipamientos" / "EDUCACION.shp"
)

In [125]:
educacion_cuenca.head()

,nombre,CATEGORIA,AREA,TIPO,SITUACION,NORm2_alum,CAPACIDAD,JERARQUIA,ADMINISTRA,geometry
0,FUNDACION NUESTROS NINOS,EDUCACION,813.708028,EDUCACION ESPECIAL,EXISTENTE,8.6,95.0,URBANO MENOR,PRIVADO,"POLYGON Z ((717323.380 9677125.366 0.000, 7173..."
1,INTEGRACION ANDINA,EDUCACION,993.719983,EDUCACION INICIAL,EXISTENTE,8.6,116.0,URBANO MENOR,PRIVADO,"POLYGON Z ((716997.092 9677032.221 0.000, 7169..."
2,LOS ANDES,EDUCACION,7751.733609,UNIDAD EDUCATIVA,EXISTENTE,8.6,901.0,URBANO MENOR,PRIVADO,"POLYGON Z ((719667.014 9678001.220 0.000, 7196..."
3,ESPIRITU DE SABIDURIA,EDUCACION,10470.483031,UNIDAD EDUCATIVA,EXISTENTE,8.6,1217.0,URBANO MENOR,PRIVADO,"POLYGON Z ((716887.171 9678395.125 0.000, 7168..."
4,CEBCI,EDUCACION,402.204868,EDUCACION INICIAL,EXISTENTE,8.6,47.0,URBANO MENOR,PRIVADO,"POLYGON Z ((723420.900 9677712.244 0.000, 7234..."


In [126]:
educacion_cuenca.SITUACION.value_counts()

EXISTENTE    404
RESERVA       12
Name: SITUACION, dtype: int64

In [127]:
educacion_cuenca = educacion_cuenca[educacion_cuenca.SITUACION=="EXISTENTE"]

In [128]:
educacion_cuenca.TIPO.value_counts()    

EDUCACION GENERAL BASICA    250
UNIDAD EDUCATIVA             82
EDUCACION INICIAL            29
UNIVERSIDAD                  19
BACHILLERATO                 14
EDUCACION ESPECIAL            7
INSTITUTO                     3
Name: TIPO, dtype: int64

Como parte del proceso de estandarización de los datos se asume que los niveles educativos disponibles en el conjunto de datos de Cuenca corresponden a los siguiente tipos de educación:


* EDUCACION GENERAL BASICA: primaria y secundaria
* UNIDAD EDUCATIVA: primaria y secundaria
* EDUCACION INICIAL: educacion inicial
* BACHILLERATO: secundaria

In [129]:
target_crs = {'init': 'epsg:4326'}
educacion_cuenca = educacion_cuenca.to_crs(target_crs)
educacion_cuenca.geometry = educacion_cuenca.geometry.centroid

/Users/cristiantorres/opt/anaconda3/envs/env_pyomu/lib/python3.8/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/var/folders/0r/zp2mz7zj28xdnvbwvnqwf8vw0000gn/T/ipykernel_14793/3343719764.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  educacion_cuenca.geometry = educacion_cuenca.geometry.centroid


In [130]:
educacion_cuenca_gb1 = educacion_cuenca[educacion_cuenca.TIPO=="EDUCACION GENERAL BASICA"].copy()
educacion_cuenca_gb2 = educacion_cuenca[educacion_cuenca.TIPO=="EDUCACION GENERAL BASICA"].copy()
educacion_cuenca_ud1 = educacion_cuenca[educacion_cuenca.TIPO=="UNIDAD EDUCATIVA"].copy()
educacion_cuenca_ud2 = educacion_cuenca[educacion_cuenca.TIPO=="UNIDAD EDUCATIVA"].copy()
educacion_cuenca_ini = educacion_cuenca[educacion_cuenca.TIPO=="EDUCACION INICIAL"].copy()
educacion_cuenca_bac = educacion_cuenca[educacion_cuenca.TIPO=="BACHILLERATO"].copy()

In [131]:
educacion_cuenca_gb1["tipo"] = "Educación Primaria"
educacion_cuenca_gb2["tipo"] = "Educación Secundaria"

educacion_cuenca_ud1["tipo"] = "Educación Primaria"
educacion_cuenca_ud2["tipo"] = "Educación Secundaria"

educacion_cuenca_ini["tipo"] = "Educación Inicial"

educacion_cuenca_bac["tipo"] = "Educación Secundaria"

In [132]:
educacion_cuenca_intermediate = pd.concat([
    educacion_cuenca_gb1,
    educacion_cuenca_gb2,
    educacion_cuenca_ud1,
    educacion_cuenca_ud2,
    educacion_cuenca_ini,
    educacion_cuenca_bac
]).reset_index(drop=True)

educacion_cuenca_intermediate = educacion_cuenca_intermediate[['nombre', 'ADMINISTRA', 'tipo', 'geometry']]

In [133]:
educacion_cuenca_intermediate.head()

,nombre,ADMINISTRA,tipo,geometry
0,AGUSTIN CUEVA TAMARIZ,PRIVADO,Educación Primaria,POINT (-78.99883 -2.91515)
1,CRISTO REY,PRIVADO,Educación Primaria,POINT (-79.00104 -2.88682)
2,CRECIENDO LIBRES,PRIVADO,Educación Primaria,POINT (-78.99676 -2.89460)
3,ARZOBISPO SERRANO,PRIVADO,Educación Primaria,POINT (-79.00535 -2.89696)
4,EL AMERICANO,PRIVADO,Educación Primaria,POINT (-79.02067 -2.88098)


In [134]:
educacion_cuenca_intermediate.head()

,nombre,ADMINISTRA,tipo,geometry
0,AGUSTIN CUEVA TAMARIZ,PRIVADO,Educación Primaria,POINT (-78.99883 -2.91515)
1,CRISTO REY,PRIVADO,Educación Primaria,POINT (-79.00104 -2.88682)
2,CRECIENDO LIBRES,PRIVADO,Educación Primaria,POINT (-78.99676 -2.89460)
3,ARZOBISPO SERRANO,PRIVADO,Educación Primaria,POINT (-79.00535 -2.89696)
4,EL AMERICANO,PRIVADO,Educación Primaria,POINT (-79.02067 -2.88098)


In [135]:
establecimiento_medellin.oferta.value_counts()

              1484
Pública        917
Privada        567
Hospitales     133
Name: oferta, dtype: int64

In [136]:
def f(x):
    if x == "PRIVADO":
        return "Privado"
    elif x == "PUBLICO":
        return "Público"
    else:
        return "Sin información"
educacion_cuenca_intermediate["oferta"] = educacion_cuenca_intermediate.ADMINISTRA.apply(f)

In [137]:
educacion_cuenca_intermediate.drop("ADMINISTRA", axis=1, inplace=True)

In [138]:
educacion_cuenca_intermediate["id"] = educacion_cuenca_intermediate.index

In [139]:
educacion_cuenca_intermediate.head()

,nombre,tipo,geometry,oferta,id
0,AGUSTIN CUEVA TAMARIZ,Educación Primaria,POINT (-78.99883 -2.91515),Privado,0
1,CRISTO REY,Educación Primaria,POINT (-79.00104 -2.88682),Privado,1
2,CRECIENDO LIBRES,Educación Primaria,POINT (-78.99676 -2.89460),Privado,2
3,ARZOBISPO SERRANO,Educación Primaria,POINT (-79.00535 -2.89696),Privado,3
4,EL AMERICANO,Educación Primaria,POINT (-79.02067 -2.88098),Privado,4


In [140]:
def f(x):
    lat = x.y
    lon = x.x
    return str(lat) + "," + str(lon)
educacion_cuenca_intermediate["destino"] = educacion_cuenca_intermediate.geometry.apply(f)

In [141]:
educacion_cuenca_intermediate = educacion_cuenca_intermediate[['id', 'destino', 'nombre', 'tipo', 'oferta', 'geometry']]

In [142]:
educacion_cuenca_intermediate.head()

,id,destino,nombre,tipo,oferta,geometry
0,0,"-2.9151467174936494,-78.99882869284582",AGUSTIN CUEVA TAMARIZ,Educación Primaria,Privado,POINT (-78.99883 -2.91515)
1,1,"-2.8868183796093705,-79.00103872073403",CRISTO REY,Educación Primaria,Privado,POINT (-79.00104 -2.88682)
2,2,"-2.8946022383986847,-78.9967601113745",CRECIENDO LIBRES,Educación Primaria,Privado,POINT (-78.99676 -2.89460)
3,3,"-2.896964603254266,-79.00534664921358",ARZOBISPO SERRANO,Educación Primaria,Privado,POINT (-79.00535 -2.89696)
4,4,"-2.880982994406767,-79.02066847136528",EL AMERICANO,Educación Primaria,Privado,POINT (-79.02067 -2.88098)


### Salud

In [196]:
salud_cuenca = gpd.read_file(
    parent_dir / "data" / "cuenca" / "equipamientos" / "SALUD.shp"
)

salud_cuenca = salud_cuenca[salud_cuenca.SITUACION=="EXISTENTE"]

In [197]:
salud_cuenca.shape

(95, 9)

In [198]:
salud_cuenca.TIPO.value_counts()

Centro Clínico-Quirúrgico Ambulatorio    12
CENTRO DE SALUD TIPO - B                 10
CENTRO DE SALUD NO DEFINIDO               9
CONSULTORIO GENERAL                       9
CENTRO DE SALUD TIPO - A                  9
CONSULTORIO DE ESPECIALIDAD (ES)          8
CLINICA                                   5
CENTRO ESPECIALIZADO                      5
CENTRO DE SALUD TIPO - C                  5
HOSPITAL GENERAL                          4
CENTRO DE SALUD NI                        4
HOSPITAL BÁSICO                           3
CENTRO DE ESPECIALIDADES                  3
HOSPITAL DE ESPECIALIDADES                2
TORRE MEDICA                              1
DIRECCION DE SALUD                        1
HOSPITAL                                  1
CENTRO DE SALUD                           1
CENTRO DE SALUD URBANO                    1
HOSPITAL ESPECIALIZADO                    1
CENTRO SIQUIATRICO                        1
Name: TIPO, dtype: int64

Como parte del proceso de estandarización de los datos se asume que los tipos disponibles en el conjunto de datos de Cuenca corresponden a los siguiente tipos de establecimiento:

* CENTRO DE SALUD TIPO - A: Atención Primaria
* CENTRO DE SALUD TIPO - B : Hospitales
* CENTRO DE SALUD TIPO - C : Hospitales
* CLINICA: Hospitales
* Centro Clínico-Quirúrgico Ambulatorio: Hospitales
* CENTRO DE SALUD NO DEFINIDO: Atención Primaria
* HOSPITAL: Hospitales
* CENTRO DE SALUD: Atención Primaria
* HOSPITAL ESPECIALIZADO: Hospitales
* HOSPITAL DE ESPECIALIDADES: Hospitales
* 'HOSPITAL BÁSICO', 'HOSPITAL GENERAL': Hospitales

In [199]:
salud_cuenca["tipo"] = "Salud"

In [200]:
salud_cuenca.head()

,NOMBRE,Shape_Leng,Shape_Area,SITUACION,CATEGORIA,TIPO,JERARQUIA,ADMINISTRA,geometry,tipo
1,HOSPITAL DE LA MUJER Y EL NIÑO,2946.554964,2946.554964,EXISTENTE,SALUD,HOSPITAL,URBANO MAYOR,PUBLICO,"POLYGON Z ((724270.163 9682983.959 0.000, 7243...",Salud
2,CLINICA ESPAÑA,1601.936241,1601.936241,EXISTENTE,SALUD,Centro Clínico-Quirúrgico Ambulatorio,URBANO MAYOR,PRIVADO,"POLYGON Z ((722904.209 9680154.664 0.000, 7229...",Salud
3,FASEC,1418.204170,1418.204170,EXISTENTE,SALUD,CENTRO ESPECIALIZADO,URBANO MAYOR,PUBLICO,"POLYGON Z ((723155.133 9677774.174 0.000, 7232...",Salud
4,CLINICA HOSP. STA. INES,2977.352029,2977.352029,EXISTENTE,SALUD,HOSPITAL GENERAL,URBANO MAYOR,PRIVADO,"POLYGON Z ((721323.242 9679003.185 0.000, 7213...",Salud
5,CENTRO MEDICO CAPULISPAMBA,2027.134261,2027.134261,EXISTENTE,SALUD,CENTRO DE SALUD NO DEFINIDO,URBANO MENOR,PUBLICO,"POLYGON Z ((729618.715 9683473.486 0.000, 7296...",Salud


In [201]:
def f(x):
    atenprimaria = ['CENTRO DE SALUD TIPO - A', 'CENTRO DE SALUD NO DEFINIDO', 'CENTRO DE SALUD']
    hospitales = ['CENTRO DE SALUD TIPO - B', 'CENTRO DE SALUD TIPO - C', 'CLINICA', 'Centro Clínico-Quirúrgico Ambulatorio', 'HOSPITAL', 'HOSPITAL ESPECIALIZADO', 'HOSPITAL DE ESPECIALIDADES', 'HOSPITAL BÁSICO', 'HOSPITAL GENERAL']
    if x in atenprimaria:
        return 'Atención Primaria'
    if x in hospitales:
        return 'Hospitales'
    return 'Sin información'

salud_cuenca['oferta'] = salud_cuenca.TIPO.apply(f)

In [205]:
salud_cuenca.shape

(95, 11)

In [206]:
salud_cuenca = salud_cuenca[salud_cuenca['oferta'] != 'Sin información']

In [207]:
salud_cuenca.reset_index(drop=True, inplace=True)
salud_cuenca["id"] = salud_cuenca.index

In [208]:
salud_cuenca.rename(columns={"NOMBRE": "nombre"}, inplace=True)

In [209]:
target_crs = {'init': 'epsg:4326'}
salud_cuenca = salud_cuenca.to_crs(target_crs)
salud_cuenca.geometry = salud_cuenca.geometry.centroid

/Users/cristiantorres/opt/anaconda3/envs/env_pyomu/lib/python3.8/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/var/folders/0r/zp2mz7zj28xdnvbwvnqwf8vw0000gn/T/ipykernel_14793/3564331871.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  salud_cuenca.geometry = salud_cuenca.geometry.centroid


In [210]:
def f(x):
    lat = x.y
    lon = x.x
    return str(lat) + "," + str(lon)
salud_cuenca["destino"] = salud_cuenca.geometry.apply(f)

In [211]:
salud_cuenca_intermediate = salud_cuenca[['id', 'destino', 'nombre', 'tipo', 'oferta', 'geometry']].copy()

In [212]:
salud_cuenca_intermediate.head()

,id,destino,nombre,tipo,oferta,geometry
0,0,"-2.8661766410417258,-78.98211986830971",HOSPITAL DE LA MUJER Y EL NIÑO,Salud,Hospitales,POINT (-78.98212 -2.86618)
1,1,"-2.8920541870747902,-78.99472787986616",CLINICA ESPAÑA,Salud,Hospitales,POINT (-78.99473 -2.89205)
2,2,"-2.902255664300106,-79.00906526214762",CLINICA HOSP. STA. INES,Salud,Hospitales,POINT (-79.00907 -2.90226)
3,3,"-2.8620042511963297,-78.93435554240325",CENTRO MEDICO CAPULISPAMBA,Salud,Atención Primaria,POINT (-78.93436 -2.86200)
4,4,"-2.9066226816711658,-79.00096619609178",CLINICA STA. ANA,Salud,Hospitales,POINT (-79.00097 -2.90662)


In [216]:
establecimientos_cuenca = pd.concat([educacion_cuenca_intermediate, salud_cuenca_intermediate]).reset_index(drop=True)

In [217]:
establecimientos_cuenca['id'] = establecimientos_cuenca.index

In [218]:
establecimientos_cuenca.head()

,id,destino,nombre,tipo,oferta,geometry
0,0,"-2.9151467174936494,-78.99882869284582",AGUSTIN CUEVA TAMARIZ,Educación Primaria,Privado,POINT (-78.99883 -2.91515)
1,1,"-2.8868183796093705,-79.00103872073403",CRISTO REY,Educación Primaria,Privado,POINT (-79.00104 -2.88682)
2,2,"-2.8946022383986847,-78.9967601113745",CRECIENDO LIBRES,Educación Primaria,Privado,POINT (-78.99676 -2.89460)
3,3,"-2.896964603254266,-79.00534664921358",ARZOBISPO SERRANO,Educación Primaria,Privado,POINT (-79.00535 -2.89696)
4,4,"-2.880982994406767,-79.02066847136528",EL AMERICANO,Educación Primaria,Privado,POINT (-79.02067 -2.88098)


In [219]:
establecimientos_cuenca.shape

(769, 6)

In [220]:
establecimientos_cuenca.tipo.value_counts()

Educación Secundaria    346
Educación Primaria      332
Salud                    62
Educación Inicial        29
Name: tipo, dtype: int64

In [221]:
establecimientos_cuenca.oferta.value_counts()

Público              541
Privado              166
Hospitales            43
Atención Primaria     19
Name: oferta, dtype: int64